<img style="float: left;padding-right: 10px" width ="40px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png">

## Semantic Kernel - Chat Completion for Decisions 

Decision Intelligence applied in this module:  
* Listing of various decision-making frameworks and with their descriptions by using Chat Completion  
* Decision Scenario: Recommendation of a decision-making framework for specific higher-stakes decisions (purchase of a car)  
* Illustrating a domain specific decision-making framework (millitary intelligence)  


In this module, the Semantic Kernel ability to create a chat experience will be introduced. This is a much richer experience than just sending simple prompts that are stateless and context is forgotten.

Semantic Kernel has first-class support for chat scenarios, where the user talks back and forth with the LLM, the arguments get populated with the history of the conversation. During each new run of the kernel, the arguments will be provided to the AI with content. This allows the LLM to know the historical context of the conversation.

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets.  
* Use the API configuration to build the Semantic Kernel orchestrator.

In [1]:
// Import the required NuGet configuration packages
#r "nuget: Microsoft.Extensions.Configuration, 9.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 9.0.0"

using Microsoft.Extensions.Configuration.Json;
using Microsoft.Extensions.Configuration;
using System.IO;

// Load the configuration settings from the local.settings.json and secrets.settings.json files
// The secrets.settings.json file is used to store sensitive information such as API keys
var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 9.0.0 Microsoft.Extensions.Configuration.Json, 9.0.0

In [2]:
// Install the required NuGet packages
// Note: This also installs the Dependency Injection Package to retrieve the ChatCompletionService
#r "nuget: Microsoft.Extensions.DependencyInjection, 9.0.0"
#r "nuget: Microsoft.SemanticKernel, 1.32.0"

using Microsoft.Extensions.DependencyInjection.Extensions;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

Kernel semanticKernel;

// Set the flag to use Azure OpenAI or OpenAI. False to use OpenAI, True to use Azure OpenAI
var useAzureOpenAI = true;

// Create a new Semantic Kernel instance
if (useAzureOpenAI)
{
    Console.WriteLine("Using Azure OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(
            deploymentName: azureOpenAIModelDeploymentName,
            endpoint: azureOpenAIEndpoint,
            apiKey: azureOpenAIAPIKey)
        .Build();
}
else
{
    Console.WriteLine("Using OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(
            modelId: openAIModelId,
            apiKey: openAIAPIKey)
        .Build();
}

var chatCompletionService = semanticKernel.Services.GetRequiredService<IChatCompletionService>();

Installed Packages Microsoft.Extensions.DependencyInjection, 9.0.0 Microsoft.SemanticKernel, 1.32.0

Using Azure OpenAI Service


### Step 2 - Execute Chat Completion with Prompt Execution Settings 

> "If you can’t describe what you are doing as a process, you don’t know what you’re doing." 
>
> -- <cite>W. Edwards Deming (renowned American engineer, statistician, and management consultant)</cite>  


Using the Semantic Kernel ChatCompletion service is very similar to inovoking a prompt for basic LLM interactions. The chat completion service will provide very similar results to invoking the prompt directly.

In [3]:
// Simple prompt to list some decision frameworks this GenAI LLM is familiar with 
// LLMs are trained on a diverse range of data and can provide insights on a wide range of topics like decision frameworks
// SLMS (smaller LLMs) are trained on a more specific range of data and may not provide insights on all topics
var simpleDecisionPrompt = """
Provide a list of 5 decision frameworks that can help improve the quality of decisions.
""";

// Create a new OpenAI prompt execution settings object
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 1000, 
    Temperature = 0.1, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };

var chatResult = await chatCompletionService.GetChatMessageContentAsync(simpleDecisionPrompt, openAIPromptExecutionSettings);

Console.WriteLine(chatResult.Content);

Certainly! Here are five decision frameworks that can help improve the quality of decisions:

1. **SWOT Analysis (Strengths, Weaknesses, Opportunities, Threats)**:
   - This framework helps in understanding the internal and external factors that can impact a decision. By analyzing strengths and weaknesses (internal factors) and opportunities and threats (external factors), decision-makers can develop strategies that leverage strengths and opportunities while mitigating weaknesses and threats.

2. **Decision Matrix (Weighted Scoring Model)**:
   - This involves listing all the options and criteria important for the decision, assigning weights to each criterion based on its importance, and scoring each option against each criterion. The weighted scores are then summed to determine the best option. This framework helps in making objective and quantifiable comparisons between different choices.

3. **Cost-Benefit Analysis (CBA)**:
   - This framework involves comparing the costs and benefi

### Step 3 - Execute Chat Completion with Streaming 

Semantic Kernel supports prompt response streaming in the ChatCompletionService. This allows responses to be streamed to the client as soon as they are made available by the LLM and API services. Below the same decision prompt is executed. However, notice that chunks are streamed and can be read by the user as soon as they appear. 

In [4]:
// Stream the chat completion response
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(simpleDecisionPrompt))
{
    Console.Write(content);
}

Certainly! Here are five decision frameworks that can help improve the quality of decisions:

1. **SWOT Analysis (Strengths, Weaknesses, Opportunities, Threats):**
   - **Strengths:** Identifies internal attributes and resources that support a successful outcome.
   - **Weaknesses:** Identifies internal attributes and resources that work against a successful outcome.
   - **Opportunities:** External factors the project can exploit to its advantage.
   - **Threats:** External factors that could cause trouble for the project.

2. **Decision Matrix Analysis (or Grid Analysis):**
   - A table used to evaluate a list of options by scoring them against a list of criteria. Each criterion is weighted based on its importance, and the options are scored accordingly to help identify the best choice.

3. **Cost-Benefit Analysis:**
   - Compares the total expected cost of each option against the total expected benefits to determine the best option in terms of financial profitability or resource eff

### Step 4 - Execute Chat Completion with Chat History 

Semantic Kernel has a ChatHistory object that can be used with the ChatCompletionService to provide historical chat context to the LLM. Notice that the ChatHistory object differentiates between the different types of chat messages:
* System Message - System or MetaPrompt. These are usually global instructions that set the "overall rules" for interacting with the LLMs.
* User Message - A message from the user
* Assistant Message - A message from the LLM. This is a message generated from an assistant or an agent. 

Identifying the messages from which role (user) it came from can help the LLM improve its own reasoning responses. This is a more sophisticated approach than passing chat history in a long dynamic string. ChatHistory objects can be serialized and persisted into databases as well.

In [5]:
// Set the overall system prompt to behave like a decision intelligence assistant (persona)
var systemPrompt = """
You are a decision intelligence assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.
""";

// Simple instruction prompt to list 5 (five) decision frameworks this GenAI LLM is familiar with
var simpleDecisionPrompt = """
Provide five Decision Frameworks that can help improve the quality of decisions.
""";

// Create a new chat history object with proper system and user message roles
var chatHistory = new ChatHistory();
chatHistory.AddSystemMessage(systemPrompt);
chatHistory.AddUserMessage(simpleDecisionPrompt);

var chatResponse = string.Empty;
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistory))
{
    chatResponse += content;
    Console.Write(content);
}
chatHistory.AddAssistantMessage(chatResponse);

Sure! Here are five Decision Frameworks that can help improve the quality of decisions:

1. **SWOT Analysis (Strengths, Weaknesses, Opportunities, Threats)**
   - **Purpose:** To assess internal and external factors related to a decision.
   - **How it Works:** 
     - Identify and list strengths intrinsic to the organization or option.
     - Identify and list weaknesses or limitations.
     - Identify and evaluate external opportunities that could be advantageous.
     - Identify and consider external threats that could pose risks.
   - **Application:** Helps in strategic planning, risk management, and choosing between options by understanding a broader context and implications.

2. **Decision Matrix (also known as Multi-Criteria Decision Analysis)**
   - **Purpose:** To evaluate and prioritize a list of options based on multiple criteria.
   - **How it Works:** 
     - List all the options to be considered.
     - Identify relevant criteria for decision-making.
     - Assign weights

In the next step, an additional prompt instruction will be added to the chat history. From the 5 decision frameworks provided in the chat history, Generative AI is asked to recommed a framework best suited for the militaty intelligence community. Notice that the previous chat history is automatically provided to provide additional intelligence context. 

> "Take time to deliberate, but when the time for action comes, stop thinking and go in."
>
> -- <cite>Napoleon Bornaparte (French Emporer and brilliant military mind)</cite>  

In [7]:
// Note: No reference is made to what previous frameworks were listed.
// Note: Previous context is maintained by the Semantic Kernel ChatHistory object 
var simpleDecisionPromptFollowupQuestionPartTwo = """
Which of these 5 decision frameworks listed above is best suited for the military intelligence community?
""";

// Add User message to the chat history
chatHistory.AddUserMessage(simpleDecisionPromptFollowupQuestionPartTwo);

var chatResponse = string.Empty;
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistory))
{
    chatResponse += content;
    Console.Write(content);
}

// Add Assistant message to the chat history
chatHistory.AddAssistantMessage(chatResponse);

For the military intelligence community, the **SWOT Analysis (Strengths, Weaknesses, Opportunities, Threats)** framework is particularly well-suited. Here's why:

1. **Comprehensive Assessment**: SWOT Analysis provides a holistic view by examining internal and external factors, which is crucial for strategic military planning and intelligence assessment.

2. **Strategic Planning**: Military intelligence often involves planning and decision-making based on both strengths and weaknesses (internal factors) as well as opportunities and threats (external factors).

3. **Risk Management**: Identifying weaknesses and threats helps in understanding potential risks and vulnerabilities, allowing for better risk mitigation strategies.

4. **Opportunity Identification**: Recognizing opportunities can help identify new tactics, technological advancements, or strategic alliances that could provide an advantage.

5. **Actionable Insights**: By thoroughly analyzing these four areas, military intellige

### Step 5 - Using Chat History in a Decision Conversation 

With Semantic Kernel maintaining state with the ChatHistory object, a user can interact with the LLM in a natural way. Note the interaction below, where the user is asking which of the particular decision frameworks are best suited to purchase a car. Without the ChatHistory being passed along, the LLM would not have had the proper context to answer the question.

> "Price is what you pay. Value is what you get."  
>
> -- <cite>Warren Buffet (one of the most successful investors of all time)</cite>  

In [6]:
// Simple prompt that asks a follow-up question about the purchase of a car
// Note: No reference is made to what previous frameworks were listed.
// Note: Previous context is maintained by the Semantic Kernel ChatHistory object 
var simpleDecisionPromptFollowupQuestion = """
Which of these 5 decision frameworks would work best for making a decision to purchase a car. 
Select only the single best one.
""";

// Add User message to the chat history
chatHistory.AddUserMessage(simpleDecisionPromptFollowupQuestion);

var chatResponse = string.Empty;
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistory))
{
    chatResponse += content;
    Console.Write(content);
}

// Add Assistant message to the chat history
chatHistory.AddAssistantMessage(chatResponse);

For making a decision to purchase a car, the **Decision Matrix (also known as Multi-Criteria Decision Analysis)** would likely work best. Here’s why:

1. **Multiple Criteria**: Buying a car involves various factors such as cost, fuel efficiency, maintenance costs, safety features, brand reliability, resale value, and more. A decision matrix allows you to consider all these criteria systematically.

2. **Weighted Importance**: You might value some factors more than others (e.g., safety might be more important to you than resale value). A decision matrix lets you assign weights to each criterion based on your priorities.

3. **Comparative Analysis**: You can compare multiple car models objectively. By scoring each model against the criteria and calculating the weighted sum, you can visualize which car offers the best overall value based on your preferences.

**How to Apply the Decision Matrix for Purchasing a Car**:

1. **List the Options**: Identify the car models you are considering.
2

### Step 6 - Semantic Kernel inspecting Decision Chat History 

The Semantic Kernel Chat History is a transparent construct that can be inspected and written out. In fact, the ChatHistory object can be manipulated to replay chats from middle interactions to simulate different outcomes.

Execute the cell below to write out the Semantic Kernel Chat History of this chat.

In [8]:
// Print the number of chat interactions and the chat history (turns)
Console.WriteLine("Number of chat interactions: " + chatHistory.Count());

foreach (var message in chatHistory)
{
    Console.WriteLine($"# {message.Role}: {message.Content}");
}

Number of chat interactions: 7
# system: You are a decision intelligence assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.
# user: Provide five Decision Frameworks that can help improve the quality of decisions.
# assistant: Certainly! Decision frameworks are structured approaches that help guide thinking and improve the quality of decisions. Here are five widely-used decision frameworks:

1. **SWOT Analysis**:
   - **Strengths**: Identify internal attributes that are helpful to achieving the objective.
   - **Weaknesses**: Identify internal attributes that are harmful to achieving the objective.
   - **Opportunities**: Identify external conditions that are helpful to achieving the objective.
   - **Threats**: Identify external conditions which could do damage to the objective.
   - **Application**: SWOT is useful for strategic planni